In [ ]:
import h5py
import matplotlib.pyplot as plt
import mpl_lego.colors as colors
import neuropacks as packs
import numpy as np
import os

from mpl_lego.ellipse import plot_cov_ellipse
from neurobiases import analysis

%matplotlib inline

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.sans-serif": ["Computer Modern Roman"]})

In [ ]:
ccycle = colors.get_default_ccycle()

In [ ]:
# Base paths
data_path = '/storage/data'
fits_path = '/storage/fits/neurobiases'
# PVC11 path
data_path = os.path.join(data_path, 'pvc11/data/spikes_gratings/data_monkey1_gratings.mat')
# Create neuropack
pvc11 = packs.PVC11(data_path=data_path)
# Fits path for PVC11-1
fits_path = os.path.join(fits_path, 'pvc11/pvc11_monkey1.h5')
fits = h5py.File(fits_path, 'r')

In [ ]:
tuning_coefs = fits['t_cosine2_ols/tuning_coefs'][0]
tuning_intercepts = fits['t_cosine2_ols/intercepts'][0]
tuning_preferences = fits['t_cosine2_ols/tuning_preferences'][0]
coupling_coefs = fits['c_lasso/coupling_coefs'][0]
coupling_coefs_srt = coupling_coefs[np.argsort(tuning_preferences)]

In [ ]:
# Figure parameters
ax_label_size = 16
ax_tick_size = 12

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))
# Choose random tuning curves
rng = np.random.default_rng(43840912)
examples = rng.integers(low=0, high=tuning_coefs.shape[0], size=len(ccycle))

for idx, example in enumerate(examples):
    angles, tc = pvc11.get_tuning_curve(
        form='cosine2',
        tuning_coefs=tuning_coefs[example],
        intercept=tuning_intercepts[example])

    ax.plot(
        angles,
        tc,
        color=ccycle[idx],
        linewidth=3,
        alpha=0.75)

ax.set_xlim([0, 330])
ax.set_ylim([-0.2, 4])       
ax.set_xticks([0, 90, 180, 270])
ax.set_yticks([0, 1, 2, 3, 4])
ax.set_xlabel(r'\textbf{Stimulus (degrees)}', fontsize=ax_label_size)
ax.set_ylabel(r'\textbf{Mean Response}', fontsize=ax_label_size)
ax.tick_params(labelsize=ax_tick_size)